# Molecular Dynamics Simulation Using Google Colab GPU
Original Method is reffered to the follwoing publications:
- Engelberger et al 2021	:https://doi.org/10.1021/acs.jchemed.1c00022
- Arantes et al 2021	:https://doi.org/10.1021/acs.jcim.1c00998.
- Lemkul et al 2021	:https://doi.org/10.33011/LIVECOMS.1.1.5068

# 01 Installation

In [ ]:
# It is recommended (and required for GROMACS 2021) to upgrade cmake
!pip install cmake --upgrade

# Download and unzip the compressed folder of GROMACS 2020.6 version
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/software/gromacs.tar.gz
!tar xzf gromacs.tar.gz

# If you get : gmx: error while loading shared libraries: libhwloc.so.5: cannot open shared object file: No such file or directory
!wget https://download.open-mpi.org/release/hwloc/v1.11/hwloc-1.11.13.tar.gz
!tar xzf hwloc-1.11.13.tar.gz
!cd /content/hwloc-1.11.13
! /content/hwloc-1.11.13/configure
!make
!sudo make install

#Installing py3Dmol using pip
!pip install py3Dmol

#Importing py3Dmol for safety
import py3Dmol

#Installing biopython using pip
!pip install biopython

In [ ]:
# Checking that our GROMACS works
%%bash
source /content/gromacs/bin/GMXRC
gmx -h

In [ ]:
#We will constantly need to source GMXRC for GROMACS to work
%%bash
source /content/gromacs/bin/GMXRC

#Try gmx here!


# 02 Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


In [ ]:
#Let's make a folder first. We need to import the os and path library
import os
from pathlib import Path

#Then, we define the path of the folder we want to create.
#Notice that the HOME folder for a hosted runtime in colab is /content/
mdpath = Path("/content/gdrive/MyDrive/works/Afumigatus-1ns")

#Now, we create the folder using the os.mkdir() command
#The if conditional is just to check whether the folder already exists
#In which case, python returns an error
if os.path.exists(mdpath):
  print("path already exists")
if not os.path.exists(mdpath):
  os.mkdir(mdpath)
  print("path was succesfully created")

In [ ]:
#First, we will change to the new folder. We will use python now :)
os.chdir(mdpath)

# 03 Cleaning up the input atomic coordinates

## 3.1 Downloading Target PDB


In [ ]:
#Importing your PDB file using biopython
import os
from Bio.PDB import *
pdbid = ['1r2g']
pdbl = PDBList()
for s in pdbid:
  pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True)
  os.rename("pdb"+s+".ent", s+".pdb")

## 3.2 Filtering residues


In [ ]:
#Here we set up a parser for our PDB
parser = PDBParser()
io=PDBIO()
structure = parser.get_structure('X', '6wzu.pdb')
#And here we set the residue conformation we want to keep
keepAltID = "A"

class KeepOneConfOnly(Select):  # Inherit methods from Select class
    def accept_atom(self, atom):
        if (not atom.is_disordered()) or atom.get_altloc() == keepAltID:
            atom.set_altloc(" ")  # Eliminate alt location ID before output.
            return True
        else:  # Alt location was not one to be output.
            return False
        # end of accept_atom()

#This will keep only conformation for each residue
io.set_structure(structure)
io.save("6wzu_ready.pdb", select=KeepOneConfOnly())
print("Your PDB was processed. Alternative side chain conformations removed!")

## 3.3 Stripping out waters and solvents


In [ ]:
#Here we set up a parser for our PDB
parser = PDBParser()
io=PDBIO()
structure = parser.get_structure('X', '6wzu_ready.pdb')
#And here we remove hydrogens, waters and ligands using Dice
io.set_structure(structure)
sel = Dice.ChainSelector('A', 1, 5000)
io.save("6wzu_clean.pdb", sel)
print("Your PDB was processed. Only the protein heavy atoms have been kept!")

## 3.4 Loading cleaned protein


In [ ]:
#First we assign the py3Dmol.view as view
view=py3Dmol.view()
#The following lines are used to add the addModel class
#to read the PDB files
view.addModel(open('6wzu_clean.pdb', 'r').read(),'pdb')
#Here we set the background color as white
view.setBackgroundColor('white')
#Here we set the visualization style and color
view.setStyle({'chain':'A'},{'cartoon': {'color':'spectrum'}})
#Centering view on all visible atoms
view.zoomTo()
#And we finally visualize the structures using the command below
view.show()

# 04 Parameterizing the atoms building up our system

Now, we will work with GROMACS to parameterize our protein, generating:

*   A **.gro** or **.pdb** coordinate file that contains all the atom types as defined by a given force field (including hydrogens).
*   A **.top** topology file containing the parameters for bonds, angles, dihedrals and non-bonded interactions defined by a given force field (potential energy function) to employ in our simulations.

1. We will parameterize our protein using the **AMBER99SB-ILDN force field** on GROMACS and obtain these files using `gmx` as shown in the code cell below. This force field is extensively used in MD simulations and has parameters that well-represent the dynamics and flexibility of folded proteins. Notice that the dynamics of highly motile proteins or intrinsically disordered regions is not the main dataset for which this force field was parameterized, and other options may better suit such goals.

In [ ]:
%%bash
source /content/gromacs/bin/GMXRC

#Using pdb2gmx to parameterize our PDB with the AMBER forcefield and SPC/E water
gmx pdb2gmx -f 6wzu_clean.pdb -o 6wzu_processed.pdb -water spce -ignh -ff amber99sb-ildn

# 05 Solvating our protein

## 5.1 Generating Periodic Box


In [ ]:
%%bash
source /content/gromacs/bin/GMXRC

#Using editconf to create a cubic box with 1.0 nm padding for our solvated system
gmx editconf -f 6wzu_processed.pdb -o 6wzu_newbox.pdb -c -d 1.0 -bt cubic

## 5.2 fill box with water


In [ ]:
%%bash
source /content/gromacs/bin/GMXRC

#Using solvate to fill up our box with water molecules
gmx solvate -cp 6wzu_newbox.pdb -o 6wzu_solv.pdb -p topol.top

## 5.3 look at the solvated system


In [ ]:
#First we assign the py3Dmol.view as view
view=py3Dmol.view()
#The following lines are used to add the addModel class
#to read the PDB files
view.addModel(open('6wzu_solv.pdb', 'r').read(),'pdb')
#Here we set the background color as white
view.setBackgroundColor('white')
#Here we set the visualization style and color
view.setStyle({'cartoon': {'color':'green'}})
#Here we add a style for showing the oxygen from water molecules
view.addStyle({'atom':'OW'},{'sphere':{'radius':'0.2'}})
#Centering the view on all visible atoms
view.zoomTo()
#And we finally visualize the structures using the command below
view.show()

# 06 Adding counterions to neutralize the global charge of the system

## 6.1 Determine absolute charge


In [ ]:
!grep "qtot" topol.top

## 6.2 download ions.mdp

In [ ]:
%%bash
wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/ions.mdp

In [ ]:
%%bash
source /content/gromacs/bin/GMXRC

#Using grompp and an MD instruction file to add counterions to our system
gmx grompp -f ions.mdp -c 6wzu_solv.pdb -p topol.top -o ions.tpr

## 6.3 run genion

In [ ]:
%%bash
source /content/gromacs/bin/GMXRC

#This is a trick to provide interactive options to gmx
echo "SOL" > options
echo " " >> options

#Using genion and the tpr to add counterions to our solvated system
gmx genion -s ions.tpr -o 6wzu_solv_ions.pdb -p topol.top -pname NA -nname CL -neutral < options

# 07 Minimizing and Equilibrating the MD system

## 7.1 Download em.mdp

In [ ]:
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/em.mdp


In [ ]:
#Check the content of the MDP file
!paste em.mdp

## 7.2 run em module

In [ ]:
%%bash
source /content/gromacs/bin/GMXRC

#Using grompp to prepare our minimization MD
gmx grompp -f em.mdp -c 6wzu_solv_ions.pdb -p topol.top -o em.tpr

#Run our minimization
gmx mdrun -v -deffnm em -nb gpu

## 7.3 potential energy

In [ ]:
%%bash
source /content/gromacs/bin/GMXRC

#This is a trick to provide interactive options to gmx
echo "Potential" > options
echo " " >> options

#Using energy to extract the potential energy of the system
gmx energy -f em.edr -o em_potential.xvg -xvg none < options

In [ ]:
#Plotting the potential energy of the system
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np

#Reading the text file containing this information
data = np.loadtxt('em_potential.xvg')

plt.title('Potential Energy during Minimization')
plt.xlabel('Energy Minimization Step')
plt.ylabel(r'E$_P$ [kJ•mol$^{-1}]$')
plt.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
plt.show()

## 7.4  download nvt.mdp

In [ ]:
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/nvt.mdp

In [ ]:
%%bash

#Using grompp to prepare our NVT equilibration MD
source /content/gromacs/bin/GMXRC
gmx grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr

In [ ]:
%%time
%%bash
#Run our NVT equilibration MD
source /content/gromacs/bin/GMXRC
gmx mdrun -deffnm nvt -nb gpu

## 7.5 check the temperature case

In [ ]:
%%bash
source /content/gromacs/bin/GMXRC

#This is a trick to provide interactive options to gmx
echo "Temperature" > options
echo " " >> options

#Using energy to extract the temperature of the system during the NVT equil MD
gmx energy -f nvt.edr -o nvt_temp.xvg -xvg none < options

In [ ]:
#Plotting the temperature of the system
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np

#Reading the text file containing this information
data = np.loadtxt('nvt_temp.xvg')

plt.title('Temperature during 1000 ps Equilibration (NVT)')
plt.xlabel('Time (ps)')
plt.ylabel('Temperature [K]')
plt.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
plt.show()

## 7.6 pressure regulation

In [ ]:
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/npt.mdp


In [ ]:
%%bash

#Using grompp to prepare our NPT equilibration MD
source /content/gromacs/bin/GMXRC
gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr

In [ ]:
%%time
%%bash
#Run our NPT equilibration MD
source /content/gromacs/bin/GMXRC
gmx mdrun -deffnm npt -nb gpu

## 7.7 NPT ensemble

In [ ]:
%%bash
source /content/gromacs/bin/GMXRC

#This is a trick to provide interactive options to gmx
echo "Pressure" > options
echo "Density" >> options
echo " "

#Using energy to extract the pressure and density of the system during the NPT equil MD
gmx energy -f npt.edr -o npt_press_dens.xvg -xvg none < options

In [ ]:
#Plotting the density of the system
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np

#Reading the text file containing this information
data = np.loadtxt('npt_press_dens.xvg')

plt.title('Pressure during 1000 ps Equilibration (NPT)')
plt.xlabel('Time (ps)')
plt.ylabel('Pressure [bar]')
plt.ylim(-500,500)

#Smoothing using Savitzky-Golay
from scipy.signal import savgol_filter
yhat = savgol_filter(data[:,1], 21, 5)

#Plot raw data and spline interpolation
plt.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
plt.plot(data[:,0], yhat, linestyle='solid', linewidth='2', color='blue')
plt.show()

In [ ]:
#Plotting the pressure of the system
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np

#Reading the text file containing this information
data = np.loadtxt('npt_press_dens.xvg')

plt.title('Density during 1000 ps Equilibration (NPT)')
plt.xlabel('Time (ps)')
plt.ylabel('Density [kg•m$^{-3}$]')
plt.ylim(1000,1025)
plt.plot(data[:,0], data[:,2], linestyle='solid', linewidth='2', color='red')
plt.show()

# 08 Obtain a production MD run and analyze the results

1. download md.mdp



In [ ]:
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/files/md.mdp


In [ ]:
%%bash

#Using grompp to prepare our production MD
source /content/gromacs/bin/GMXRC
gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md_1.tpr

In [ ]:
#Check the content of the production MD file
!paste md.mdp

In [ ]:
%%time
%%bash
#Run our production MD
source /content/gromacs/bin/GMXRC
gmx mdrun -deffnm md_1 -nb gpu

# 09 Appending Simulaiton after interruption

In [ ]:
%%time
%%bash
#Append
source /content/gromacs/bin/GMXRC
gmx mdrun -v -cpi md_1.cpt -noappend -deffnm md_1 -gpu_id 0


# 10 Concatenate all xtc files

In [ ]:
%%time
%%bash
#Concatenation of xtc files
source /content/gromacs/bin/GMXRC
gmx trjcat -f md_1.xtc md_1.part0002.xtc md_1.part0003.xtc md_1.part0004.xtc md_1.part0005.xtc md_1.part0006.xtc md_1.part0007.xtc md_1.part0008.xtc md_1.part0009.xtc md_1.part0010.xtc md_1.part0011.xtc md_1.part0012.xtc md_1.part0013.xtc -o md_1_all.xtc


## 10.1 Check whether the xtc files have been properly concatenated or not.

In [ ]:
%%time
%%bash
#Check whether the xtc files have been properly concatenated or not
source /content/gromacs/bin/GMXRC
gmx check -f md_1_all.xtc
